# 02 - Batch Data Collection

Efficiently collect data for multiple Stadium project candidates.

**Goals:**
1. Load Stadium candidates from `data/stadium_candidates.md`
2. Quick verification of maintainer counts
3. Batch collection with rate limit management
4. Progress tracking and error handling

## Setup

In [3]:
import os
import sys
import json
import time
from pathlib import Path
from datetime import datetime

import pandas as pd
from dotenv import load_dotenv

# Add src to path
sys.path.insert(0, '../src')
from collection.github_collector import GitHubCollector

# Load environment from .env file
env_path = Path("../.env")
if env_path.exists():
    load_dotenv(env_path)
    print(f"✅ Loaded .env from {env_path.resolve()}")
else:
    load_dotenv()
    print("⚠️  No .env file found, trying default locations")

GITHUB_TOKEN = os.getenv('GITHUB_TOKEN')

if not GITHUB_TOKEN:
    raise ValueError(
        "GITHUB_TOKEN not found!\n"
        "1. Copy .env.example to .env: cp ../.env.example ../.env\n"
        "2. Edit .env and add your GitHub token\n"
        "3. Get a token at: https://github.com/settings/tokens"
    )

if GITHUB_TOKEN == "your_github_token_here":
    raise ValueError(
        "GITHUB_TOKEN is still the placeholder!\n"
        "Edit ../.env and replace with your actual token"
    )

# Initialize collector
collector = GitHubCollector(token=GITHUB_TOKEN)

print("✅ Setup complete!")
rate = collector.get_rate_limit()
print(f"   Rate limit: {rate['core']['remaining']}/{rate['core']['limit']}")

/Users/ibrahimcesar/Dev/categories-of-the-commons/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


✅ Loaded .env from /Users/ibrahimcesar/Dev/categories-of-the-commons/.env
✅ Setup complete!
   Rate limit: 5000/5000


## 1. Define Stadium Candidates

Based on research criteria:
- High usage/downloads
- Few maintainers (≤3 ideal, or high dominance)
- Critical infrastructure packages

In [5]:
# Import from centralized candidate lists
import sys
sys.path.insert(0, '../data')
from candidates import (
    STADIUM_ALL, STADIUM_COLLECTED, STADIUM_HIGH_PRIORITY,
    FEDERATION_CANDIDATES, CLUB_CANDIDATES, TOY_CANDIDATES,
    get_uncollected, print_status
)

# Show collection status across all categories
print_status()

# Use the centralized Stadium candidates
all_candidates = [{"repo": repo, "ecosystem": "mixed"} for repo in STADIUM_ALL]

print(f"\nTotal Stadium candidates: {len(all_candidates)}")
print(f"Already collected: {len(STADIUM_COLLECTED)}")
print(f"Remaining: {len(get_uncollected('stadium'))}")

CANDIDATE COLLECTION STATUS
STADIUM       12/ 37 (32%)
FEDERATION     0/ 19 (0%)
CLUB           0/ 19 (0%)
TOY            1/  1 (100%)

Total Stadium candidates: 37
Already collected: 12
Remaining: 25


## 2. Quick Verification - Check Maintainer Counts

Before full collection, quickly verify candidates meet Stadium criteria.

In [6]:
def quick_verify(repo_name: str) -> dict:
    """Quick verification of Stadium criteria."""
    try:
        # Get basic metrics
        metrics = collector.collect_repository_metrics(repo_name)
        maintainers = collector.collect_maintainer_data(repo_name)
        
        # Get top contributor dominance
        contributors = collector.collect_contributor_data(repo_name, max_contributors=10)
        
        dominance = 0
        if contributors:
            total = sum(c['contributions'] for c in contributors)
            if total > 0:
                dominance = contributors[0]['contributions'] / total * 100
        
        return {
            "repo": repo_name,
            "stars": metrics.get('stargazers_count', 0),
            "language": metrics.get('language', 'Unknown'),
            "active_maintainers": maintainers['statistics'].get('active_maintainers_6mo', 0),
            "top_contributor": contributors[0]['login'] if contributors else 'N/A',
            "top_contributor_pct": round(dominance, 1),
            "stadium_likely": maintainers['statistics'].get('active_maintainers_6mo', 0) <= 3 or dominance > 40,
            "error": None
        }
    except Exception as e:
        return {
            "repo": repo_name,
            "error": str(e)
        }

In [7]:
# Quick verify all candidates (uses ~50-100 API calls per repo)
print("Quick verification of Stadium candidates...")
print("="*70)

verification_results = []

for i, candidate in enumerate(all_candidates):
    repo = candidate['repo']
    print(f"[{i+1}/{len(all_candidates)}] Checking {repo}...", end=" ")
    
    result = quick_verify(repo)
    result['ecosystem'] = candidate['ecosystem']
    verification_results.append(result)
    
    if result.get('error'):
        print(f"❌ Error: {result['error'][:50]}")
    elif result.get('stadium_likely'):
        print(f"✅ Stadium likely ({result['active_maintainers']} maintainers, {result['top_contributor_pct']}% dominance)")
    else:
        print(f"⚠️  Maybe not Stadium ({result['active_maintainers']} maintainers, {result['top_contributor_pct']}% dominance)")
    
    # Rate limit check
    if (i + 1) % 5 == 0:
        rate = collector.get_rate_limit()
        print(f"    [Rate limit: {rate['core']['remaining']}/{rate['core']['limit']}]")
        if rate['core']['remaining'] < 500:
            print("⚠️  Rate limit low, pausing...")
            time.sleep(60)

print("\n" + "="*70)
print("Verification complete!")

Quick verification of Stadium candidates...
[1/37] Checking curl/curl... 

Request GET /repos/curl/curl/collaborators failed with 403: Forbidden


Could not fetch collaborators (requires admin access): Must have push access to view repository collaborators.: 403 {"message": "Must have push access to view repository collaborators.", "documentation_url": "https://docs.github.com/rest/collaborators/collaborators#list-repository-collaborators", "status": "403"}
Will use MAINTAINERS.md/CONTRIBUTORS.md files instead...


KeyboardInterrupt: 

In [8]:
# Display verification results
df_verify = pd.DataFrame(verification_results)

# Filter successful verifications
df_success = df_verify[df_verify['error'].isna()].copy()

print(f"\nSuccessfully verified: {len(df_success)}/{len(df_verify)}")
print(f"Stadium likely: {df_success['stadium_likely'].sum()}")
print(f"Uncertain: {(~df_success['stadium_likely']).sum()}")

# Display table
display_cols = ['repo', 'ecosystem', 'stars', 'active_maintainers', 'top_contributor', 'top_contributor_pct', 'stadium_likely']
df_success[display_cols].sort_values('stadium_likely', ascending=False)

KeyError: 'error'

## 3. Select Confirmed Stadium Projects

In [9]:
# Use centralized uncollected list
# This uses the COLLECTED list from stadium_candidates.py
uncollected = get_uncollected("stadium")

# Also allow collecting high priority first
confirmed_stadium = STADIUM_HIGH_PRIORITY + [r for r in uncollected if r not in STADIUM_HIGH_PRIORITY]

print(f"Confirmed Stadium projects to collect ({len(confirmed_stadium)}):")
for repo in confirmed_stadium[:10]:  # Show first 10
    priority = "HIGH" if repo in STADIUM_HIGH_PRIORITY else ""
    print(f"  - {repo} {priority}")
if len(confirmed_stadium) > 10:
    print(f"  ... and {len(confirmed_stadium) - 10} more")

Confirmed Stadium projects to collect (25):
  - uuidjs/uuid HIGH
  - debug-js/debug HIGH
  - npm/node-semver HIGH
  - vercel/ms HIGH
  - node-fetch/node-fetch HIGH
  - yargs/yargs HIGH
  - urllib3/urllib3 HIGH
  - dateutil/dateutil HIGH
  - certifi/python-certifi HIGH
  - pallets/click HIGH
  ... and 15 more


## 4. Batch Collection - Full Dataset

Collect complete data for confirmed Stadium projects.

In [10]:
def collect_with_retry(repo_name: str, since_days: int = 365, max_retries: int = 3) -> dict:
    """Collect data with retry logic."""
    for attempt in range(max_retries):
        try:
            data = collector.collect_complete_dataset(repo_name, since_days=since_days)
            return {"success": True, "data": data, "error": None}
        except Exception as e:
            if "rate limit" in str(e).lower():
                print(f"      Rate limit hit, waiting 60s...")
                time.sleep(60)
            elif attempt < max_retries - 1:
                print(f"      Retry {attempt + 1}/{max_retries}...")
                time.sleep(5)
            else:
                return {"success": False, "data": None, "error": str(e)}
    return {"success": False, "data": None, "error": "Max retries exceeded"}

In [11]:
# Check which projects already have data
data_dir = Path("../data/raw")
existing_files = {f.stem.replace('_data', '').replace('_', '/'): f for f in data_dir.glob("*_data.json")}

print(f"Existing data files: {len(existing_files)}")
for repo in existing_files:
    print(f"  ✓ {repo}")

# Filter to only collect missing ones
to_collect = [repo for repo in confirmed_stadium if repo not in existing_files]
print(f"\nNeed to collect: {len(to_collect)}")

Existing data files: 23
  ✓ benjaminp/six
  ✓ glennrp/libpng
  ✓ curl/curl
  ✓ psf/requests
  ✓ yaml/pyyaml
  ✓ sindresorhus/got
  ✓ debug-js/debug
  ✓ uuidjs/uuid
  ✓ vercel/ms
  ✓ tj/commander.js
  ✓ dateutil/dateutil
  ✓ yargs/yargs
  ✓ pallets/click
  ✓ ibrahimcesar/react-lite-youtube-embed
  ✓ zloirock/core-js
  ✓ chalk/chalk
  ✓ certifi/python-certifi
  ✓ node-fetch/node-fetch
  ✓ serde-rs/serde
  ✓ axios/axios
  ✓ npm/node-semver
  ✓ urllib3/urllib3
  ✓ madler/zlib

Need to collect: 15


In [12]:
# Batch collection
collection_results = []
start_time = datetime.now()

print(f"Starting batch collection for {len(to_collect)} projects...")
print("="*70)

for i, repo in enumerate(to_collect):
    print(f"\n[{i+1}/{len(to_collect)}] Collecting {repo}...")
    
    # Check rate limit before starting
    rate = collector.get_rate_limit()
    if rate['core']['remaining'] < 500:
        wait_time = 60
        print(f"    ⏳ Rate limit low ({rate['core']['remaining']}), waiting {wait_time}s...")
        time.sleep(wait_time)
    
    # Collect data
    result = collect_with_retry(repo, since_days=365)
    
    if result['success']:
        # Save to file
        output_path = data_dir / f"{repo.replace('/', '_')}_data.json"
        collector.save_data(result['data'], output_path)
        
        stars = result['data']['repository'].get('stargazers_count', 0)
        contributors = len(result['data']['contributors'])
        print(f"    ✅ Success! ({stars:,} stars, {contributors} contributors)")
        
        collection_results.append({
            "repo": repo,
            "success": True,
            "stars": stars,
            "contributors": contributors,
            "file": str(output_path)
        })
    else:
        print(f"    ❌ Failed: {result['error'][:50]}")
        collection_results.append({
            "repo": repo,
            "success": False,
            "error": result['error']
        })

elapsed = datetime.now() - start_time
print("\n" + "="*70)
print(f"Batch collection complete! Time: {elapsed}")

Starting batch collection for 15 projects...

[1/15] Collecting rust-lang/regex...


1/7 Collecting repository metrics...
2/7 Identifying maintainers...


Request GET /repos/rust-lang/regex/collaborators failed with 403: Forbidden


Could not fetch collaborators (requires admin access): Must have push access to view repository collaborators.: 403 {"message": "Must have push access to view repository collaborators.", "documentation_url": "https://docs.github.com/rest/collaborators/collaborators#list-repository-collaborators", "status": "403"}
Will use MAINTAINERS.md/CONTRIBUTORS.md files instead...
3/7 Collecting contributor data...


Contributors: 100it [00:02, 42.16it/s]


4/7 Collecting commit history...


...

4/7 Collecting commit history...


Commits: 307it [03:22,  1.52it/s]


5/7 Collecting pull request data...


6/7 Collecting issue data...


7/7 Checking governance files...

Collection complete for: sparklemotion/nokogiri

Data saved to ../data/raw/sparklemotion_nokogiri_data.json
    ✅ Success! (6,214 stars, 100 contributors)

Batch collection complete! Time: 1:22:36.816233


In [13]:
# Summary
df_results = pd.DataFrame(collection_results)

print("\n" + "="*60)
print("COLLECTION SUMMARY")
print("="*60)

if len(df_results) > 0:
    success_count = df_results['success'].sum()
    print(f"Successful: {success_count}/{len(df_results)}")
    print(f"Failed: {len(df_results) - success_count}")
    
    if 'stars' in df_results.columns:
        df_success = df_results[df_results['success']]
        print(f"\nTotal stars collected: {df_success['stars'].sum():,}")
        print(f"Total contributors: {df_success['contributors'].sum():,}")

# Show all collected data
all_data_files = list(data_dir.glob("*_data.json"))
print(f"\nTotal data files: {len(all_data_files)}")
total_size = sum(f.stat().st_size for f in all_data_files) / 1024
print(f"Total size: {total_size:.1f} KB")


COLLECTION SUMMARY
Successful: 15/15
Failed: 0

Total stars collected: 330,694
Total contributors: 1,400

Total data files: 38
Total size: 7628.5 KB


## 5. Final Rate Limit Check

In [14]:
rate = collector.get_rate_limit()
print(f"\n📊 Final Rate Limit Status:")
print(f"   Core API: {rate['core']['remaining']}/{rate['core']['limit']} remaining")
print(f"   Resets at: {rate['core']['reset']}")


📊 Final Rate Limit Status:
   Core API: 2899/5000 remaining
   Resets at: 2025-11-29 03:35:07+00:00


## Next Steps

1. Run `01_data_exploration.ipynb` to analyze collected data
2. Run `03_statistical_analysis.ipynb` for hypothesis testing
3. Add Federation/Club control projects for comparison